# Modeling an autoregressive process

## Defining the autoregressive process

#### An autoregressive process establishes that the output variable depends linearly on its own previous values. In other words, it is a regression of the variable against itself.

#### An autoregressive process is denoted as an AR(p) process, where p is the order. In such a process, the present value yt is a linear combination of a constant C, the present error term ϵt, which is also white noise, and the past values of the series yt–p. The magnitude of the influence of the past values on the present value is denoted as ϕp, which represents the coefficients of the AR(p)) model. Mathematically, we express a general AR(p) model with equation 5.1.

- yt = C + ϕ1yt–1 + ϕ2yt–2 +⋅⋅⋅ ϕpyt–p + ϵt

## Autoregressive process

#### An autoregressive process is a regression of a variable against itself. In a time series, this means that the present value is linearly dependent on its past values.